# 02_train_resnet50.ipynb

**Goal:** Evaluate the trained baseline ResNet-50 model using ROC, AUC, precision–recall, and confusion matrix visualizations.

In [ ]:
# Import necessary libraries

import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve, average_precision_score
from src.model import PneumoniaResNet50
from src.data_loader import get_data_loader
from pathlib import Path


In [ ]:
# Load the trained model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = Path("saved_models/resnet50_best.pt")

model = PneumoniaResNet50()
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

csv_path = "data/rsna_subset/train_labels_subset.csv"
img_dir = "data/rsna_subset/train_images"
loader = get_data_loader(csv_path, img_dir, batch_size=8)
